<a href="https://colab.research.google.com/github/Sartorio12/Projeto_Modulo_5/blob/main/notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando bibliotecas pandas e nunmpy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Limpeza inicial dos dados

Aqui vamos fazer uma limpeza e pré processamento inicial dos dados. Em uma análise preliminar encontrei uma série de problemas e vou realizar as seguintes correções.
<ol>
    <li>Remover registros onde existe incosistencia de reembolso e glosa </li>
    <li>Remover registros sem partido e sem estado</li>
    <li>Agrupar os nomes de partidos que se alteraram (tb posso checar pelo numero)</li>    
    <ol>
            <li>PMDB virou MDB</li>
            <li>PEN virou PATRIOTAS (Se fundiu com o PRP em 2018 e deixou de usar a sigla PATRI em 2019)</li>
            <li>PP e PP** estao se referindo ao mesmo partido</li>        
            <li>PPS virou CIDADANIA </li>
            <li>PRB virou REPUBLICANOS (verificar deputados REPUBLICANOS antes do mandato de 2018)</li>
            <li>PR virou PL</li>
    </ol>
</ol>

Glossário das colunas:
- Nome Parlamentar(txNomeParlamentar) :  *Nome adotado pelo Parlamentar ao tomar posse do seu mandato.*
- ideCadastro(Identificador Único do Parlamentar) : *Número que identifica unicamente um deputado federal na CD.*
- nuCarteiraParlamentar(Número da Carteira Parlamentar) : *Documento usado para identificar um deputado federal na CD.*
- nuLegislatura(Número da  Legislatura) : *No contexto da cota CEAP, representa o ano base de início da legislatura.*
- sgUF(Sigla da Unidade Federal) : *Representa a UF do parlamentar.*
- sdPartido(Sigla do Partido) : *Representa o partido do parlamentar.*
- codLegislatura(Código da Legislatura) : *No contexto da cota CEAP, o seu conteúdo representa o código identificador da Legislatura.*
- numSubCota(Número da Subcota) : *Representa o código do Tipo de Despesa, comprovada por meio da emissão de um documento fiscal, a qual é debitada na cota do deputado.*
- txtDescricao(Descrição da Subcota) : *Descrição do Tipo de Despesa.*
- numEspecificacaoSubCota(Número da Especificação da Subcota) : *Representa o código desta especificação mais detalhada.*
- txtDescricaoEspecificacao(Descrição da Especificação da Subcota) : *Detalha o tipo de despesa efetuada.*
- txtFornecedor(Fornecedor) : *Representa o fornecedor do produto ou serviço presente no documento fiscal.*
- txtCNPJCPF(CNPJ/CPF) : *Indica se o fornecedor é CNPJ ou CPF , presente no documento fiscal.*
- txtNumero(Número do Documento) : *Indica o número do documento do emissor da NF em questão.*
- indTipoDocumento(Indicativo de Tipo de Documento Fiscal) : *Tipo de documento do fiscal – 0 para Nota Fiscal; 1 para Recibo; 2 para Despesa no Exterior.*
- datEmissao(Data de Emissão) : *A data de emissão do documento.*
- vlrDocumento(Valor do Documento) : *O seu conteúdo é o valor de face do documento fiscal ou o valor do documento que deu causa à despesa.*
- vlrGlosa(Valor da Glosa) : *Valor da glosa do documento que deu causa à despesa.*
- vlrLiquido(Valor Liquido) : *Valor líquido do documento (será calculado pela diferença entre o Valor do Documento e o Valor da Glosa)*
- numMes(Mês) : *Indica o mês da emissão da nota.*
- numAno(Ano) : *Indica o ano da emissão da nota.*
- numParcela(Número da Parcela) : *O seu conteúdo representa o número da parcela do documento fiscal. Ocorre quando o documento tem de ser reembolsado de forma parcelada.*
- txtPassageiro(Nome do Passageiro) : *O nome do passageiro, em caso da emissão do documento ser uma passagem aérea.*
- txtTrecho(Trecho) : *Representa qual o trecho da viagem, somente em casos de viagem aérea.*
- numLote(Número do Lote) : *Representa a capa do lote em que esse documento será incluído para ser entregue na Câmara para ressarcimento.*
- numRessarcimento(Número do Ressarcimento) : *Indica o ressarcimento do qual o documento faz parte.*
- vlrRestituição(Valor da Restituição) : *Indica o valor que será restituído sobre aquele documento.*
- nuDeputadoId(Identificador do Solicitante) : *Número que identifica um Parlamentar ou Liderança na Transparência da Cota para Exercício da Atividade Parlamentar.*

In [2]:
# lendo os arquivos e numero de linhas e colunas
df1 = pd.read_csv('Ano-2019.csv', sep=';')
df1.shape

(288992, 31)

In [3]:
df2 = pd.read_csv('Ano-2020.csv', sep=';')
df2.shape

(160903, 31)

In [4]:
df3 = pd.read_csv('Ano-2021.csv', sep=';')
df3.shape

(105605, 31)

In [5]:
#Concatenando os 3 arquivos em um único
df_unico = pd.concat([df1,df2,df3])
df_unico.shape

(555500, 31)

In [23]:
df_unico.sgPartido.value_counts()

PT               71008
PSL              67028
PP               44060
PL               38495
MDB              38487
PSD              36623
PSB              34907
PSDB             34371
REPUBLICANOS     29028
PDT              28706
DEM              27015
PTB              12719
SOLIDARIEDADE    12350
PODE             10574
CIDADANIA         9341
AVANTE            9257
PROS              8599
PSOL              8099
PCdoB             7288
PSC               6991
PATRIOTA          5951
NOVO              5829
PV                3761
PR                1118
REDE               708
PRB                531
S.PART.            377
PPS                225
PATRI              154
PHS                139
PPL                 20
Name: sgPartido, dtype: int64

In [6]:
# Criando uma cópia do dataframe como boa prática
df_copia = df_unico.copy()
df_copia

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1554246,NaN,NaN,2812,6738121,http://camara.leg.br/cota-parlamentar/nota-fis...
1,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1570078,NaN,NaN,2812,6769606,http://camara.leg.br/cota-parlamentar/nota-fis...
2,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1570503,NaN,NaN,2812,6771099,http://camara.leg.br/cota-parlamentar/nota-fis...
3,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1579375,NaN,NaN,2812,6793690,http://camara.leg.br/cota-parlamentar/nota-fis...
4,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1590980,NaN,NaN,2812,6821674,http://camara.leg.br/cota-parlamentar/nota-fis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105600,Henrique do Paraíso,3.470995e+10,217036.0,588.0,2019,SP,REPUBLICANOS,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2021,0,NaN,NaN,1781785,NaN,NaN,3486,7228378,https://www.camara.leg.br/cota-parlamentar/doc...
105601,Henrique do Paraíso,3.470995e+10,217036.0,588.0,2019,SP,REPUBLICANOS,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2021,0,NaN,NaN,1785880,NaN,NaN,3486,7237484,https://www.camara.leg.br/cota-parlamentar/doc...
105602,Henrique do Paraíso,3.470995e+10,217036.0,588.0,2019,SP,REPUBLICANOS,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2021,0,NaN,NaN,1782000,NaN,NaN,3486,7228945,https://www.camara.leg.br/cota-parlamentar/doc...
105603,Henrique do Paraíso,3.470995e+10,217036.0,588.0,2019,SP,REPUBLICANOS,56,10,TELEFONIA,...,2021,0,NaN,NaN,0,0.0,NaN,3486,0,NaN


In [7]:
# Nome das colunas
df_copia.columns

Index(['txNomeParlamentar', 'cpf', 'ideCadastro', 'nuCarteiraParlamentar',
       'nuLegislatura', 'sgUF', 'sgPartido', 'codLegislatura', 'numSubCota',
       'txtDescricao', 'numEspecificacaoSubCota', 'txtDescricaoEspecificacao',
       'txtFornecedor', 'txtCNPJCPF', 'txtNumero', 'indTipoDocumento',
       'datEmissao', 'vlrDocumento', 'vlrGlosa', 'vlrLiquido', 'numMes',
       'numAno', 'numParcela', 'txtPassageiro', 'txtTrecho', 'numLote',
       'numRessarcimento', 'vlrRestituicao', 'nuDeputadoId', 'ideDocumento',
       'urlDocumento'],
      dtype='object')

In [8]:
# Tipo de dado de cada coluna
df_copia.dtypes

txNomeParlamentar             object
cpf                          float64
ideCadastro                  float64
nuCarteiraParlamentar        float64
nuLegislatura                  int64
sgUF                          object
sgPartido                     object
codLegislatura                 int64
numSubCota                     int64
txtDescricao                  object
numEspecificacaoSubCota        int64
txtDescricaoEspecificacao     object
txtFornecedor                 object
txtCNPJCPF                    object
txtNumero                     object
indTipoDocumento               int64
datEmissao                    object
vlrDocumento                 float64
vlrGlosa                     float64
vlrLiquido                   float64
numMes                         int64
numAno                         int64
numParcela                     int64
txtPassageiro                 object
txtTrecho                     object
numLote                        int64
numRessarcimento             float64
v

In [9]:
# Vendo todas as informações
df_copia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555500 entries, 0 to 105604
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   txNomeParlamentar          555500 non-null  object 
 1   cpf                        553661 non-null  float64
 2   ideCadastro                553759 non-null  float64
 3   nuCarteiraParlamentar      553759 non-null  float64
 4   nuLegislatura              555500 non-null  int64  
 5   sgUF                       553759 non-null  object 
 6   sgPartido                  553759 non-null  object 
 7   codLegislatura             555500 non-null  int64  
 8   numSubCota                 555500 non-null  int64  
 9   txtDescricao               555500 non-null  object 
 10  numEspecificacaoSubCota    555500 non-null  int64  
 11  txtDescricaoEspecificacao  136177 non-null  object 
 12  txtFornecedor              555467 non-null  object 
 13  txtCNPJCPF                 53

observação: fizemos a consulta e achamos alguns valores nulos em algumas colunas mas achamos que elas não são relevantes pras consultas do nosso projeto.
è preciso analisar em grupo se devemos removê-las ou mantê-las.

In [10]:
# Procurando numero de dados nulos em percentual
round(df_copia.isnull().mean()*100,2)

txNomeParlamentar             0.00
cpf                           0.33
ideCadastro                   0.31
nuCarteiraParlamentar         0.31
nuLegislatura                 0.00
sgUF                          0.31
sgPartido                     0.31
codLegislatura                0.00
numSubCota                    0.00
txtDescricao                  0.00
numEspecificacaoSubCota       0.00
txtDescricaoEspecificacao    75.49
txtFornecedor                 0.01
txtCNPJCPF                    4.28
txtNumero                     0.00
indTipoDocumento              0.00
datEmissao                    2.82
vlrDocumento                  0.00
vlrGlosa                      0.00
vlrLiquido                    0.00
numMes                        0.00
numAno                        0.00
numParcela                    0.00
txtPassageiro                76.69
txtTrecho                    76.88
numLote                       0.00
numRessarcimento             67.45
vlrRestituicao               99.93
nuDeputadoId        

In [11]:
# o describe foi feito pra destacar os valores negativos presentes nas colunas:'vlrDocumento' e 'vlrLiquido' 
df_copia.describe()

,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,codLegislatura,numSubCota,numEspecificacaoSubCota,indTipoDocumento,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento
count,5.536610e+05,553759.000000,553759.000000,555500.000000,555500.000000,555500.000000,555500.000000,555500.000000,555500.000000,555500.000000,555500.000000,555500.000000,555500.000000,555500.000000,5.555000e+05,180842.0,364.000000,555500.000000,5.555000e+05
mean,3.208493e+10,166015.342644,289.007700,2018.894387,55.973597,250.656994,0.251131,1.029525,884.412172,3.900387,808.744693,6.083879,2019.669870,0.000238,1.130421e+06,0.0,873.633571,2734.361870,5.045073e+06
std,2.921625e+10,45864.510704,153.129661,0.641326,0.160331,412.069770,0.457949,1.567946,2464.526292,139.446699,2464.253989,3.337029,0.775475,0.015413,7.872364e+05,0.0,2367.571616,773.342999,2.865350e+06
min,1.608657e+06,4930.000000,1.000000,2015.000000,55.000000,1.000000,0.000000,0.000000,-6605.570000,0.000000,-7821.200000,1.000000,2019.000000,0.000000,0.000000e+00,0.0,1.000000,19.000000,0.000000e+00
25%,6.197278e+09,141531.000000,160.000000,2019.000000,56.000000,3.000000,0.000000,0.000000,63.000000,0.000000,49.510000,3.000000,2019.000000,0.000000,0.000000e+00,0.0,6.975000,2265.000000,1.705442e+06
50%,2.227515e+10,178938.000000,292.000000,2019.000000,56.000000,10.000000,0.000000,0.000000,201.380000,0.000000,184.500000,6.000000,2019.000000,0.000000,1.620008e+06,0.0,26.105000,3062.000000,6.888214e+06
75%,5.098035e+10,204449.000000,424.000000,2019.000000,56.000000,122.000000,0.000000,1.000000,747.600000,0.000000,642.000000,9.000000,2020.000000,0.000000,1.707915e+06,0.0,277.395000,3312.000000,7.075930e+06
max,9.989866e+10,217330.000000,627.000000,2019.000000,56.000000,999.000000,4.000000,4.000000,172000.000000,86000.000000,151260.000000,12.000000,2021.000000,1.000000,1.789366e+06,0.0,20000.000000,3487.000000,7.245337e+06


observação: Achamos importante explicar o que são esses valores negativos ( compensação de passagens aéreas) conforme dicionário do arquivo.
https://www2.camara.leg.br/transparencia/cota-para-exercicio-da-atividade-parlamentar/explicacoes-sobre-o-formato-dos-arquivos-xml

In [12]:
# Visualizando as 5 primeiras linhas do dataframe
df_copia.head()

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1554246,NaN,NaN,2812,6738121,http://camara.leg.br/cota-parlamentar/nota-fis...
1,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1570078,NaN,NaN,2812,6769606,http://camara.leg.br/cota-parlamentar/nota-fis...
2,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1570503,NaN,NaN,2812,6771099,http://camara.leg.br/cota-parlamentar/nota-fis...
3,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1579375,NaN,NaN,2812,6793690,http://camara.leg.br/cota-parlamentar/nota-fis...
4,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2019,0,NaN,NaN,1590980,NaN,NaN,2812,6821674,http://camara.leg.br/cota-parlamentar/nota-fis...


In [13]:
# Visualizando as cinco últimas linhas do dataframe
df_copia.tail()

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
105600,Henrique do Paraíso,3.470995e+10,217036.0,588.0,2019,SP,REPUBLICANOS,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2021,0,NaN,NaN,1781785,NaN,NaN,3486,7228378,https://www.camara.leg.br/cota-parlamentar/doc...
105601,Henrique do Paraíso,3.470995e+10,217036.0,588.0,2019,SP,REPUBLICANOS,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2021,0,NaN,NaN,1785880,NaN,NaN,3486,7237484,https://www.camara.leg.br/cota-parlamentar/doc...
105602,Henrique do Paraíso,3.470995e+10,217036.0,588.0,2019,SP,REPUBLICANOS,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2021,0,NaN,NaN,1782000,NaN,NaN,3486,7228945,https://www.camara.leg.br/cota-parlamentar/doc...
105603,Henrique do Paraíso,3.470995e+10,217036.0,588.0,2019,SP,REPUBLICANOS,56,10,TELEFONIA,...,2021,0,NaN,NaN,0,0.0,NaN,3486,0,NaN
105604,DaLua do Rota,5.117259e+10,217330.0,590.0,2019,AP,PSC,56,10,TELEFONIA,...,2021,0,NaN,NaN,0,0.0,NaN,3487,0,NaN


In [14]:
# Buscamos amostra aleatória de cinco linhas
df_copia.sample(5)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
5595,Benedita da Silva,3.629333e+10,73701.0,291.0,2019,RJ,PT,56,10,TELEFONIA,...,2020,0,NaN,NaN,0,0.0,NaN,188,0,NaN
92090,Bohn Gass,1.255821e+10,160538.0,499.0,2019,RS,PT,56,3,COMBUSTÍVEIS E LUBRIFICANTES.,...,2019,0,NaN,NaN,1650320,NaN,NaN,2405,6956851,https://www.camara.leg.br/cota-parlamentar/doc...
9508,Ivan Valente,3.765558e+10,73531.0,359.0,2019,SP,PSOL,56,999,PASSAGEM AÉREA - RPA,...,2019,0,Ivan Valente,BSB/CGH,0,0.0,NaN,1049,1721901,NaN
92691,Carlos Chiodini,5.031909e+08,204361.0,473.0,2019,SC,MDB,56,10,TELEFONIA,...,2020,0,NaN,NaN,1739262,NaN,NaN,3331,7141622,NaN
88745,Tito,7.176248e+10,197438.0,214.0,2019,BA,AVANTE,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2020,0,NaN,NaN,1740716,NaN,NaN,3417,7144763,https://www.camara.leg.br/cota-parlamentar/doc...


In [15]:
# Essa consulta fizemos um novo df, filtrando na coluna 'txtDescricao' valores diferentes de 'PASSAGEM AÉREA - RPA'  e 'PASSAGEM AÉREA - SIGEPA' e a partir desse novo df fazer os cálculos de despesas.
valores = df_copia[(df_copia['txtDescricao'] != 'PASSAGEM AÉREA - RPA')&(df_copia['txtDescricao'] != 'PASSAGEM AÉREA - SIGEPA')]
valores['txtDescricao'].unique()
valores.vlrLiquido

0          111.30
1          117.19
2          120.15
3          151.18
4          119.90
           ...   
105600    5000.00
105601    7000.00
105602    4000.00
105603       1.10
105604       1.08
Name: vlrLiquido, Length: 427074, dtype: float64

In [16]:
# os dois valores negativos da coluna 'VrlLiquido'
valores.query('vlrLiquido < 0')['vlrLiquido']

23210    -868.00
28211   -1762.66
Name: vlrLiquido, dtype: float64

In [17]:
# Convertemos a coluna datEmissao de object para datatime e colocamos só pra mostrar o formato dia/mes/ano pq o horário é irrelevante para as consultas
df_copia['datEmissao'] = pd.to_datetime(df_copia['datEmissao'], format = '%Y/%m/%d')
df_copia['datEmissao']

0        2019-01-16
1        2019-02-04
2        2019-02-25
3        2019-03-28
4        2019-05-07
            ...    
105600   2021-07-28
105601   2021-08-11
105602   2021-07-29
105603          NaT
105604          NaT
Name: datEmissao, Length: 555500, dtype: datetime64[ns]

In [18]:
#Conferindo se o type da data foi transformado
df_copia.dtypes

txNomeParlamentar                    object
cpf                                 float64
ideCadastro                         float64
nuCarteiraParlamentar               float64
nuLegislatura                         int64
sgUF                                 object
sgPartido                            object
codLegislatura                        int64
numSubCota                            int64
txtDescricao                         object
numEspecificacaoSubCota               int64
txtDescricaoEspecificacao            object
txtFornecedor                        object
txtCNPJCPF                           object
txtNumero                            object
indTipoDocumento                      int64
datEmissao                   datetime64[ns]
vlrDocumento                        float64
vlrGlosa                            float64
vlrLiquido                          float64
numMes                                int64
numAno                                int64
numParcela                      

In [19]:
df_copia.shape

(555500, 31)

In [20]:
# df_copia[df_copia['cpf'].isnull()]
df_copia.isnull().sum()


txNomeParlamentar                 0
cpf                            1839
ideCadastro                    1741
nuCarteiraParlamentar          1741
nuLegislatura                     0
sgUF                           1741
sgPartido                      1741
codLegislatura                    0
numSubCota                        0
txtDescricao                      0
numEspecificacaoSubCota           0
txtDescricaoEspecificacao    419323
txtFornecedor                    33
txtCNPJCPF                    23790
txtNumero                         0
indTipoDocumento                  0
datEmissao                    15650
vlrDocumento                      0
vlrGlosa                          0
vlrLiquido                        0
numMes                            0
numAno                            0
numParcela                        0
txtPassageiro                426007
txtTrecho                    427074
numLote                           0
numRessarcimento             374658
vlrRestituicao              

In [21]:
df_copia['txNomeParlamentar'].str.contains('LIDMIN', na=False)

0         False
1         False
2         False
3         False
4         False
          ...  
105600    False
105601    False
105602    False
105603    False
105604    False
Name: txNomeParlamentar, Length: 555500, dtype: bool

# A fazer:
- Preencher valores NaN com valores padrões (Em alguns campos, não temos CPF por exemplo, por se tratar de uma secretaria pedindo verba para algo)
- 

### Análise dos DataFrames ( Harão e Jonathas)